In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, numpy, pickle
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import matplotlib


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'



/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
Using Theano backend.


In [2]:
# Write out mutation counts
mutations = pandas.read_csv("../data/derived/annotated_mutations.with_mhc_binders.csv.bz2",
                            converters={'binding_peptides': lambda x: eval(x) if x else {}})
mutations

/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,source_id,donor,genome,contig,interbase_start,interbase_end,ref,alt,sources,effect,gene,context_5_prime,context_3_prime,context_mutation,binding_affinity,binding_allele,dna_alt_reads,dna_ref_reads,dna_total_reads,rna_alt_reads,rna_ref_reads,rna_total_reads,context_mutation_3p5p,binding_peptides,num_binders,indel,snv,mnv,confident,vaf,any_alt_reads,mutation_id,site_id,unique_to_treated
0,AOCS-001-1-7,AOCS-001,GRCh37,14,24774226,24774227,C,A,annotated_mutations,silent,NOP9,ACCTTCCTAAAGCGG,GAGAGGCTTGGGAAC,C>A,NaN,NaN,17.0,28.0,45.0,185.0,244.0,429.0,G(C>A)G,{},0.0,False,True,False,True,0.377777,True,14:24774226 C>A,14:24774226,False
1,AOCS-001-1-7,AOCS-001,GRCh37,X,19968971,19968972,T,C,annotated_mutations,p.I548M,CXorf23,GTCATTTGGATCTAT,ATTTTGATCAGAGTC,T>C,94.33,B*41:01,16.0,51.0,67.0,0.0,77.0,77.0,T(T>C)A,"{'TLIKMIDP': 34863.83, 'IKMIDPNDLRH': 35670.23...",5.0,False,True,False,True,0.238806,True,X:19968971 T>C,X:19968971,False
2,AOCS-001-1-7,AOCS-001,GRCh37,8,35406835,35406836,G,A,annotated_mutations,p.E44K,UNC5D,CTGATGGGATGGATT,GGGAAGGGCTTCGCC,C>T,80.84,A*11:01,14.0,39.0,53.0,0.0,0.0,0.0,T(C>T)G,"{'PKSIPSAPGT': 43270.03, 'DNGEALPKS': 44344.5,...",4.0,False,True,False,True,0.264150,True,8:35406835 G>A,8:35406835,False
3,AOCS-001-1-7,AOCS-001,GRCh37,7,47872830,47872831,A,G,annotated_mutations,p.L2065P,HUS1 PKD1L1,CTGCATCAGCCATTC,CTCTGGGAGTGGCAG,T>C,343.29,A*11:01,16.0,31.0,47.0,0.0,1.0,1.0,C(T>C)C,"{'ASAIPSGS': 15126.14, 'QPASAIPSG': 41737.86, ...",2.0,False,True,False,True,0.340425,True,7:47872830 A>G,7:47872830,False
4,AOCS-001-1-7,AOCS-001,GRCh37,17,17721666,17721667,G,C,annotated_mutations,p.R394G,SREBF1,AAATCTGCTGTCTTG,GCAAGGCCATCGACT,C>G,43.42,C*15:02,12.0,22.0,34.0,200.0,54.0,254.0,G(C>G)G,"{'NKSAVLGKA': 24205.73, 'KLNKSAVLG': 23291.86,...",2.0,False,True,False,True,0.352940,True,17:17721666 G>C,17:17721666,False
5,AOCS-001-1-7,AOCS-001,GRCh37,7,148169063,148169064,G,T,annotated_mutations,intergenic,None,CAAAGAAGGGCCACC,AATCTAAACCAGCAC,C>A,NaN,NaN,19.0,39.0,58.0,0.0,0.0,0.0,C(C>A)A,{},0.0,False,True,False,True,0.327586,True,7:148169063 G>T,7:148169063,False
6,AOCS-001-1-7,AOCS-001,GRCh37,1,107152783,107152784,G,A,annotated_mutations,intergenic,None,TCCACTGTGTTTGTT,ATCCCTTACAAAAGA,C>T,NaN,NaN,14.0,69.0,83.0,0.0,0.0,0.0,T(C>T)A,{},0.0,False,True,False,True,0.168674,True,1:107152783 G>A,1:107152783,False
7,AOCS-001-1-7,AOCS-001,GRCh37,2,142133268,142133269,T,A,annotated_mutations,intronic,LRP1B,GCATTAAATTCCCAA,ATTGCACTTTGGTCC,T>A,NaN,NaN,9.0,28.0,37.0,0.0,0.0,0.0,A(T>A)A,{},0.0,False,True,False,True,0.243243,True,2:142133268 T>A,2:142133268,False
8,AOCS-001-1-7,AOCS-001,GRCh37,2,143687576,143687577,G,A,annotated_mutations,intronic,KYNU,GCCCTACAAAGCCTT,TGTTGTGAGGGAAAA,C>T,NaN,NaN,11.0,55.0,66.0,0.0,0.0,0.0,T(C>T)T,{},0.0,False,True,False,True,0.166666,True,2:143687576 G>A,2:143687576,False
9,AOCS-001-1-7,AOCS-001,GRCh37,1,144017729,144017730,G,T,annotated_mutations,non-coding-transcript,SRGAP2B,TGTAAGTGCACCAAT,GACACTCTGTATCTA,C>A,NaN,NaN,26.0,158.0,184.0,0.0,0.0,0.0,T(C>A)G,{},0.0,False,True,False,True,0.141304,True,1:144017729 G>T,1:144017729,False


In [3]:
sources = pandas.read_csv("../data/derived/sources.full.csv", index_col="source_id")
sources["source_id"] = sources.index
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

,RNA_ID,RNA biospecimen,DNA_id,DNA_biospecimen,specimen_type_description,CollectionPoint,SpecimenType,RNA data file,DNA_sample_string,icgc_sample_id,project_code_x,submitted_sample_id,icgc_specimen_id,submitted_specimen_id_x,icgc_donor_id_x,submitted_donor_id_x,analyzed_sample_interval,percentage_cellularity_x,level_of_cellularity_x,study,project_code_y,study_specimen_involved_in,submitted_specimen_id_y,icgc_donor_id_y,submitted_donor_id_y,specimen_type,specimen_type_other,specimen_interval,specimen_donor_treatment_type,specimen_donor_treatment_type_other,specimen_processing,specimen_processing_other,specimen_storage,specimen_storage_other,tumour_confirmed,specimen_biobank,specimen_biobank_id,specimen_available,tumour_histological_type,tumour_grading_system,tumour_grade,tumour_grade_supplemental,tumour_stage_system,tumour_stage,tumour_stage_supplemental,digital_image_of_stained_section,percentage_cellularity_y,level_of_cellularity_y,cohort,donor,tissue_type,timepoint,treated,metastasis,interval_days,bam_path_tumor_dna,bam_path_tumor_rna,source_id,treated_paired
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,ICGCDBDE20130916001,AOCS-001-2-0,AOCS_001_ICGC_DBPC_20130205_002,AOCS-001-1-7,primary tumour,Primary,Tumour,130906_D81P8DQ1_0153_C2704ACXX.nopd.AOCS_001_I...,ICGCDBPC20130205002,SA505244,OV-AU,AOCS-001-1-7,SP101515,AOCS-001-1,DO46325,AOCS-001,1895.0,73.0,61-80%,PCAWG,OV-AU,PCAWG,AOCS-001-1,DO46325,AOCS-001,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,73.0,61-80%,AOCS,AOCS-001,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/EGAZ00001018...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,AOCS-001-1-7,False
AOCS-004-1-5,ICGCDBDE20130916003,AOCS-004-2-9,AOCS_004_ICGC_DBPC_20130205_004,AOCS-004-1-5,primary tumour,Primary,Tumour,130906_D81P8DQ1_0157_C270DACXX.nopd.AOCS_004_I...,ICGCDBPC20130205004,SA505270,OV-AU,AOCS-004-1-5,SP101519,AOCS-004-1,DO46327,AOCS-004,1350.0,77.0,61-80%,PCAWG,OV-AU,PCAWG,AOCS-004-1,DO46327,AOCS-004,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,77.0,61-80%,AOCS,AOCS-004,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,AOCS-004-1-5,False
AOCS-005-1-8,ICGCDBDE20130916004,AOCS-005-2-1,AOCS_005_ICGC_DBPC_20130205_006,AOCS-005-1-8,primary tumour,Primary,Tumour,130906_D81P8DQ1_0157_C270DACXX.nopd.AOCS_005_I...,ICGCDBPC20130205006,SA505282,OV-AU,AOCS-005-1-8,SP101521,AOCS-005-1,DO46328,AOCS-005,2128.0,90.0,>81%,PCAWG,OV-AU,PCAWG,AOCS-005-1,DO46328,AOCS-005,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8441/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,90.0,>81%,AOCS,AOCS-005,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,AOCS-005-1-8,False
AOCS-034-1-0,ICGCDBLG2010050402TR,AOCS-034-2-4,AOCS_034_ICGC_DBPC_20130205_009,AOCS-034-1-0,primary tumour,Primary,Tumour,131206_EXTERN_0045_AC2KJBACXX.nopd.AOCS_034_IC...,ICGCDBPC20130205009,SA505297,OV-AU,AOCS-034-1-0,SP101523,AOCS-034-1,DO46329,AOCS-034,1953.0,84.0,>81%,PCAWG,OV-AU,PCAWG,AOCS-034-1,DO46329,AOCS-034,Primary tumour - solid tissue,NaN,0,no treatment,NaN,fresh,NaN,"frozen, vapor phase",NaN,no,NaN,NaN,NaN,8140/3,Silverberg,3.0,NaN,FIGO,III,NaN,NaN,84.0,>81%,AOCS,AOCS-034,solid,primary,False,False,0,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,/demeter/scratch/datasets/ega-tim/_EGAZ0000101...,AOCS-034-1-0,True
AOCS-034-3-8,ICGCDBDE20131122024,AOCS-034-4-1,AOCS_034_ICGC_DBPC_20130205_007,AOCS-034-3-8,recurrent ascitic fluid,Recurrence,Ascites,131206_EXTERN_0045_AC2KJBACXX.nopd.AOCS_034_IC...,ICGCDBPC20130205007,SA505303,OV-AU,AOCS-034-3-8,SP101524,AOCS-034-13,DO46329,AOCS-034,110.0,99.0,>81%,NaN,OV-AU,N

In [4]:
mutations.iloc[0]

source_id                       AOCS-001-1-7
donor                               AOCS-001
genome                                GRCh37
contig                                    14
interbase_start                     24774226
interbase_end                       24774227
ref                                        C
alt                                        A
sources                  annotated_mutations
effect                                silent
gene                                    NOP9
context_5_prime              ACCTTCCTAAAGCGG
context_3_prime              GAGAGGCTTGGGAAC
context_mutation                         C>A
binding_affinity                         NaN
binding_allele                           NaN
dna_alt_reads                             17
dna_ref_reads                             28
dna_total_reads                           45
rna_alt_reads                            185
rna_ref_reads                            244
rna_total_reads                          429
context_mu

In [5]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-034-3-8,69,85,7,65,72,44,16,86,80,33,57,72,41,30,21,60,76,25,38,42,23,5,39,35,84,61,10,120,43,38,12,82,70,285,37,265,47,45,55,123,36,44,37,59,19,16,37,29,68,77,5,44,46,49,5,72,82,74,43,68,13,18,22,32,35,32,25,41,12,12,22,19,98,76,10,99,330,82,18,492,267,97,41,184,37,33,25,56,49,39,15,38,32,18,37,69,new
AOCS-064-3-3,145,157,11,178,127,61,32,139,172,82,65,133,79,74,79,127,108,69,73,145,50,23,66,69,120,105,13,125,78,58,25,80,81,82,58,194,128,103,127,219,56,118,85,258,34,44,75,193,152,151,9,91,62,157,6,117,124,145,41,105,54,50,41,104,51,91,42,85,17,49,49,80,78,98,10,114,100,93,18,182,86,66,36,120,84,73,42,186,96,111,37,129,51,38,70,160,new
AOCS-065-3-6,26,23,3,30,24,15,5,43,33,18,24,26,17,10,15,32,26,10,11,27,9,4,14,8,28,22,3,27,24,12,5,28,30,44,21,48,18,21,13,28,6,13,15,22,7,2,13,24,27,15,1,20,9,17,1,27,29,30,24,27,9,14,15,10,14,12,1,18,11,4,12,12,28,27,2,26,33,17,1,38,25,30,16,25,7,18,10,21,21,10,6,25,18,7,13,20,new
AOCS-086-3-2,97,125,6,78,83,42,23,93,82,56,87,95,44,29,41,59,85,34,64,93,32,11,33,29,90,79,11,128,62,49,21,88,126,362,50,316,112,103,130,189,58,64,64,109,23,20,44,54,116,127,13,79,46,118,9,115,119,109,64,107,16,30,24,39,47,53,41,48,16,13,30,25,73,71,8,100,122,76,11,207,107,81,34,104,71,39,25,77,60,61,31,72,34,14,48,77,new
AOCS-088-3-8,204,230,17,204,134,78,27,126,189,98,96,198,78,64,83,144,122,84,73,143,43,18,74,49,175,174,21,201,89,73,19,117,123,138,67,232,111,143,137,234,63,135,59,86,24,30,74,73,126,115,15,149,61,71,11,91,139,89,77,131,62,74,69,141,56,51,42,63,35,16,64,53,208,154,21,260,327,154,32,514,413,189,45,320,97,93,61,207,90,99,45,98,46,38,94,87,new
AOCS-091-3-0,139,166,15,121,167,63,22,121,159,74,122,128,64,69,52,96,138,71,112,125,31,18,55,37,138,127,12,173,81,44,12,128,83,165,92,192,133,120,175,221,64,84,68,114,31,30,61,51,98,145,16,87,63,91,7,128,156,103,92,104,45,47,48,80,65,76,53,89,22,22,58,40,110,100,9,127,121,92,11,213,107,86,50,107,97,74,50,140,100,85,49,100,52,30,56,80,new
AOCS-092-3-3,75,161,2,88,45,17,5,42,53,25,52,56,41,52,28,44,42,28,32,49,23,10,21,34,112,77,21,202,22,24,10,64,149,540,46,522,315,172,268,361,54,43,48,106,21,19,37,30,82,154,6,68,18,111,3,64,109,106,48,58,22,27,15,29,23,73,19,28,8,23,12,18,57,67,8,111,54,32,8,133,81,91,37,114,41,28,24,48,27,35,17,41,9,16,29,42,new
AOCS-093-3-6,56,81,4,56,48,24,13,41,49,34,29,57,27,12,38,40,66,34,44,53,10,8,21,20,37,48,4,65,30,46,15,55,56,127,26,128,83,58,91,81,34,41,42,57,11,14,33,25,53,70,8,32,22,59,7,51,69,54,24,48,15,17,29,22,26,37,21,28,7,8,14,10,49,37,7,64,89,60,6,129,101,53,17,74,30,16,19,46,42,34,21,46,24,18,25,42,new
AOCS-094-6-X,96,105,10,102,97,62,27,106,119,54,67,129,44,42,45,90,93,60,74,105,35,10,43,24,95,94,6,87,61,52,18,95,57,44,50,111,48,66,62,99,47,108,54,65,15,30,58,29,48,60,9,68,42,32,9,55,71,49,43,67,29,26,25,52,49,45,40,47,21,7,39,31,99,56,9,96,176,85,19,237,95,60,28,103,71,37,39,87,81,110,46,113,38,25,58,75,new


In [6]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


0


context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,121.0,85.0,15.0,92.0,84.0,48.0,12.0,83.0,93.0,53.0,123.0,105.0,30.0,32.0,44.0,60.0,125.0,48.0,81.0,120.0,24.0,15.0,32.0,32.0,99.0,86.0,13.0,81.0,52.0,40.0,12.0,72.0,72.0,75.0,74.0,110.0,44.0,60.0,72.0,68.0,35.0,70.0,49.0,44.0,17.0,16.0,25.0,31.0,65.0,69.0,8.0,56.0,42.0,42.0,5.0,40.0,89.0,76.0,83.0,67.0,29.0,19.0,42.0,54.0,54.0,29.0,47.0,67.0,4.0,5.0,34.0,27.0,78.0,94.0,19.0,103.0,149.0,89.0,12.0,235.0,158.0,109.0,64.0,127.0,51.0,51.0,28.0,91.0,67.0,47.0,37.0,81.0,22.0,21.0,35.0,63.0,all
AOCS-004-1-5,109.0,108.0,17.0,83.0,78.0,47.0,12.0,72.0,138.0,68.0,131.0,100.0,54.0,57.0,63.0,83.0,155.0,59.0,103.0,155.0,23.0,15.0,34.0,33.0,125.0,79.0,13.0,133.0,57.0,51.0,26.0,79.0,99.0,77.0,103.0,132.0,36.0,44.0,56.0,78.0,85.0,80.0,85.0,118.0,22.0,31.0,46.0,93.0,50.0,44.0,9.0,56.0,39.0,33.0,8.0,47.0,90.0,79.0,123.0,108.0,39.0,34.0,39.0,48.0,93.0,61.0,75.0,110.0,17.0,12.0,31.0,39.0,67.0,77.0,10.0,117.0,59.0,68.0,15.0,147.0,142.0,102.0,57.0,125.0,80.0,42.0,42.0,104.0,112.0,93.0,41.0,95.0,27.0,28.0,62.0,69.0,all
AOCS-005-1-8,107.0,66.0,6.0,60.0,55.0,38.0,18.0,53.0,123.0,55.0,167.0,101.0,49.0,26.0,32.0,71.0,125.0,57.0,100.0,127.0,32.0,17.0,37.0,53.0,56.0,44.0,9.0,45.0,43.0,32.0,14.0,56.0,84.0,59.0,143.0,85.0,23.0,35.0,32.0,52.0,43.0,51.0,45.0,73.0,22.0,18.0,31.0,59.0,66.0,43.0,10.0,53.0,25.0,18.0,10.0,31.0,77.0,67.0,134.0,62.0,24.0,11.0,13.0,31.0,50.0,25.0,33.0,50.0,25.0,10.0,43.0,35.0,85.0,83.0,21.0,117.0,148.0,74.0,11.0,252.0,206.0,102.0,91.0,178.0,61.0,14.0,19.0,66.0,58.0,50.0,37.0,80.0,36.0,20.0,44.0,84.0,all
AOCS-034-1-0,254.0,211.0,27.0,173.0,203.0,107.0,33.0,199.0,253.0,119.0,215.0,182.0,100.0,71.0,77.0,161.0,279.0,94.0,116.0,163.0,71.0,23.0,72.0,75.0,191.0,175.0,27.0,243.0,120.0,104.0,36.0,202.0,175.0,426.0,143.0,432.0,101.0,113.0,129.0,212.0,95.0,108.0,95.0,170.0,39.0,52.0,98.0,80.0,148.0,177.0,16.0,130.0,104.0,110.0,17.0,171.0,199.0,159.0,165.0,165.0,54.0,53.0,70.0,93.0,103.0,67.0,94.0,95.0,49.0,48.0,306.0,67.0,220.0,230.0,27.0,233.0,535.0,190.0,30.0,816.0,492.0,229.0,130.0,353.0,110.0,74.0,59.0,165.0,150.0,120.0,52.0,136.0,78.0,40.0,86.0,173.0,all
AOCS-034-3-8,254.0,211.0,27.0,173.0,203.0,107.0,33.0,199.0,253.0,119.0,215.0,182.0,100.0,71.0,77.0,161.0,279.0,94.0,116.0,163.0,71.0,23.0,72.0,75.0,191.0,175.0,27.0,243.0,120.0,104.0,36.0,202.0,175.0,426.0,143.0,432.0,101.0,113.0,129.0,212.0,95.0,108.0,95.0,170.0,39.0,52.0,98.0,80.0,148.0,177.0,16.0,130.0,104.0,110.0,17.0,171.0,199.0,159.0,165.0,165.0,54.0,53.0,70.0,93.0,103.0,67.0,94.0,95.0,49.0,48.0,306.0,67.0,220.0,230.0,27.0,233.0,535.0,190.0,30.0,816.0,492.0,229.0,130.0,353.0,110.0,74.0,59.0,165.0,150.0,120.0,52.0,136.0,78.0,40.0,86.0,173.0,all
AOCS-055-1-7,152.0,122.0,17.0,137.0,147.0,78.0,42.0,149.0,184.0,94.0,318.0,144.0,55.0,51.0,63.0,115.0,247.0,109.0,168.0,285.0,43.0,8.0,57.0,64.0,130.0,91.0,14.0,120.0,124.0,90.0,38.0,147.0,126.0,113.0,183.0,203.0,50.0,53.0,76.0,101.0,103.0,140.0,96.0,155.0,39.0,48.0,82.0,121.0,89.0,76.0,19.0,70.0,65.0,59.0,15.0,109.0,140.0,97.0,222.0,107.0,41.0,39.0,43.0,76.0,117.0,73.0,89.0,140.0,34.0,24.0,71.0,62.0,263.0,152.0,27.0,197.0,1617.0,385.0,97.0,1656.0,1

In [7]:
# STATEMENT_UNIQUE_TO_TREATED_COUNT
print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].shape)

print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors)
].shape)

(132117, 34)
(303074, 34)


In [8]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/derived/mutation_contexts_counts.csv", index=False)
muts_r

,source_id,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,A[C>T]G,A[C>T]T,A[T>A]A,A[T>A]C,A[T>A]G,A[T>A]T,A[T>C]A,A[T>C]C,A[T>C]G,A[T>C]T,A[T>G]A,A[T>G]C,A[T>G]G,A[T>G]T,C[C>A]A,C[C>A]C,C[C>A]G,C[C>A]T,C[C>G]A,C[C>G]C,C[C>G]G,C[C>G]T,C[C>T]A,C[C>T]C,C[C>T]G,C[C>T]T,C[T>A]A,C[T>A]C,C[T>A]G,C[T>A]T,C[T>C]A,C[T>C]C,C[T>C]G,C[T>C]T,C[T>G]A,C[T>G]C,C[T>G]G,C[T>G]T,G[C>A]A,G[C>A]C,G[C>A]G,G[C>A]T,G[C>G]A,G[C>G]C,G[C>G]G,G[C>G]T,G[C>T]A,G[C>T]C,G[C>T]G,G[C>T]T,G[T>A]A,G[T>A]C,G[T>A]G,G[T>A]T,G[T>C]A,G[T>C]C,G[T>C]G,G[T>C]T,G[T>G]A,G[T>G]C,G[T>G]G,G[T>G]T,T[C>A]A,T[C>A]C,T[C>A]G,T[C>A]T,T[C>G]A,T[C>G]C,T[C>G]G,T[C>G]T,T[C>T]A,T[C>T]C,T[C>T]G,T[C>T]T,T[T>A]A,T[T>A]C,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1-7,AOCS-001-1-7,121.0,85.0,15.0,92.0,84.0,48.0,12.0,83.0,93.0,53.0,123.0,105.0,30.0,32.0,44.0,60.0,125.0,48.0,81.0,120.0,24.0,15.0,32.0,32.0,99.0,86.0,13.0,81.0,52.0,40.0,12.0,72.0,72.0,75.0,74.0,110.0,44.0,60.0,72.0,68.0,35.0,70.0,49.0,44.0,17.0,16.0,25.0,31.0,65.0,69.0,8.0,56.0,42.0,42.0,5.0,40.0,89.0,76.0,83.0,67.0,29.0,19.0,42.0,54.0,54.0,29.0,47.0,67.0,4.0,5.0,34.0,27.0,78.0,94.0,19.0,103.0,149.0,89.0,12.0,235.0,158.0,109.0,64.0,127.0,51.0,51.0,28.0,91.0,67.0,47.0,37.0,81.0,22.0,21.0,35.0,63.0,all
AOCS-004-1-5,AOCS-004-1-5,109.0,108.0,17.0,83.0,78.0,47.0,12.0,72.0,138.0,68.0,131.0,100.0,54.0,57.0,63.0,83.0,155.0,59.0,103.0,155.0,23.0,15.0,34.0,33.0,125.0,79.0,13.0,133.0,57.0,51.0,26.0,79.0,99.0,77.0,103.0,132.0,36.0,44.0,56.0,78.0,85.0,80.0,85.0,118.0,22.0,31.0,46.0,93.0,50.0,44.0,9.0,56.0,39.0,33.0,8.0,47.0,90.0,79.0,123.0,108.0,39.0,34.0,39.0,48.0,93.0,61.0,75.0,110.0,17.0,12.0,31.0,39.0,67.0,77.0,10.0,117.0,59.0,68.0,15.0,147.0,142.0,102.0,57.0,125.0,80.0,42.0,42.0,104.0,112.0,93.0,41.0,95.0,27.0,28.0,62.0,69.0,all
AOCS-005-1-8,AOCS-005-1-8,107.0,66.0,6.0,60.0,55.0,38.0,18.0,53.0,123.0,55.0,167.0,101.0,49.0,26.0,32.0,71.0,125.0,57.0,100.0,127.0,32.0,17.0,37.0,53.0,56.0,44.0,9.0,45.0,43.0,32.0,14.0,56.0,84.0,59.0,143.0,85.0,23.0,35.0,32.0,52.0,43.0,51.0,45.0,73.0,22.0,18.0,31.0,59.0,66.0,43.0,10.0,53.0,25.0,18.0,10.0,31.0,77.0,67.0,134.0,62.0,24.0,11.0,13.0,31.0,50.0,25.0,33.0,50.0,25.0,10.0,43.0,35.0,85.0,83.0,21.0,117.0,148.0,74.0,11.0,252.0,206.0,102.0,91.0,178.0,61.0,14.0,19.0,66.0,58.0,50.0,37.0,80.0,36.0,20.0,44.0,84.0,all
AOCS-034-1-0,AOCS-034-1-0,254.0,211.0,27.0,173.0,203.0,107.0,33.0,199.0,253.0,119.0,215.0,182.0,100.0,71.0,77.0,161.0,279.0,94.0,116.0,163.0,71.0,23.0,72.0,75.0,191.0,175.0,27.0,243.0,120.0,104.0,36.0,202.0,175.0,426.0,143.0,432.0,101.0,113.0,129.0,212.0,95.0,108.0,95.0,170.0,39.0,52.0,98.0,80.0,148.0,177.0,16.0,130.0,104.0,110.0,17.0,171.0,199.0,159.0,165.0,165.0,54.0,53.0,70.0,93.0,103.0,67.0,94.0,95.0,49.0,48.0,306.0,67.0,220.0,230.0,27.0,233.0,535.0,190.0,30.0,816.0,492.0,229.0,130.0,353.0,110.0,74.0,59.0,165.0,150.0,120.0,52.0,136.0,78.0,40.0,86.0,173.0,all
AOCS-034-3-8,AOCS-034-3-8,254.0,211.0,27.0,173.0,203.0,107.0,33.0,199.0,253.0,119.0,215.0,182.0,100.0,71.0,77.0,161.0,279.0,94.0,116.0,163.0,71.0,23.0,72.0,75.0,191.0,175.0,27.0,243.0,120.0,104.0,36.0,202.0,175.0,426.0,143.0,432.0,101.0,113.0,129.0,212.0,95.0,108.0,95.0,170.0,39.0,52.0,98.0,80.0,148.0,177.0,16.0,130.0,104.0,110.0,17.0,171.0,199.0,159.0,165.0,165.0,54.0,53.0,70.0,93.0,103.0,67.0,94.0,95.0,49.0,48.0,306.0,67.0,220.0,230.0,27.0,233.0,535.0,190.0,30.0,816.0,492.0,229.0,130.0,353.0,110.0,74.0,59.0,165.0,150.0,120.0,52.0,136.0,78.0,40.0,86.0,173.0,all
AOCS-055-1-7,AOCS-055-1-7,152.0,122.0,17.0,137.0,147.0,78.0,42.0,149.0,184.0,94.0,318.0,144.0,55.0,51.0,63.0,115.0,247.0,109.0,168.0,285.0,43.0,8.0,57.0,64.0,130.0,91.0,14.0,120.0,124.0,90.0,38.0,147.0,126.0,113.0,183.0,203.0,50.0,53.0,76.0,101.0,103.0,140.0,96.0,155.0,39.0,48.0,82.0,121.0,89.0,76.0,19.0,70.0,65.0,59.0,15.0,109.0,140.0,97.0,222.0,107.0,41.0,39.0,43.0,76.0,117.0,73.0,89.0,140.0,3